# Scraping TopShop product images
***

### 0. Import necessary libraries

In [17]:
import numpy as np
import time
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd
import urllib
import cv2
from selenium import webdriver
!pip install webdriver_manager
from webdriver_manager.chrome import ChromeDriverManager

### 1. Get image link for all products

In [138]:
url = "http://us.topshop.com/en/tsus/category/bags-accessories-7594012/bags/N-avtZqjzZdgm?No=0&Nrpp=20&siteId=%2F13052"
# I used Firefox; you can use whichever browser you like.
browser = webdriver.Chrome(ChromeDriverManager().install())

# Tell Selenium to get the URL you're interested in.
browser.get(url)

# Selenium script to grab all product info on page -> wait 3 seconds for the next batch of data to load, 
# then continue scrolling. It will continue to do this until the page stops loading new data.
all_ = []
links = []
desc = []
prices = []

lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
match=False
while(match==False):
    lastCount = lenOfPage
    time.sleep(3)
    elements = (browser.find_elements_by_class_name(name= 'product_image'))
    all_.extend(elements)
    links.extend([e.get_attribute('src') for e in elements])
    desc.extend([e.get_attribute('data-title') for e in elements])
    elements2 = (browser.find_elements_by_class_name(name= 'product_price'))
    prices.extend([e.text for e in elements2])
    lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    
    if lastCount==lenOfPage:
        match=True
    
# Now that the page is fully scrolled, grab the source code.
source_data = browser.page_source
browser.close()


Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/gracejeong/.wdm/chromedriver/2.46/mac64/chromedriver


In [139]:
links

['https://images.topshop.com/i/TopShop/TS24P05QMON_M_1.jpg?$Large$',
 'https://images.topshop.com/i/TopShop/TS24R14QNAT_M_1.jpg?$Large$',
 'https://images.topshop.com/i/TopShop/TS24I13QCHT_M_1.jpg?$Large$',
 'https://images.topshop.com/i/TopShop/TS24I14QMON_M_1.jpg?$Large$',
 'https://images.topshop.com/i/TopShop/TS24I08QRED_M_1.jpg?$Large$',
 'https://images.topshop.com/i/TopShop/TS24W15QCLR_M_1.jpg?$Large$',
 'https://images.topshop.com/i/TopShop/TS24B03QBLK_M_1.jpg?$Large$',
 'https://images.topshop.com/i/TopShop/TS24B03QNAT_M_1.jpg?$Large$',
 'https://images.topshop.com/i/TopShop/TS24B32QNAT_M_1.jpg?$Large$',
 'https://images.topshop.com/i/TopShop/TS63Q30QMUL_M_1.jpg?$Large$',
 'https://images.topshop.com/i/TopShop/TS63H91PMUL_M_1.jpg?$Large$',
 'https://images.topshop.com/i/TopShop/TS63Q23QMUL_F_1.jpg?$Large$',
 'https://images.topshop.com/i/TopShop/TS63H90PMUL_M_1.jpg?$Large$',
 'https://images.topshop.com/i/TopShop/TS63Q12QPNK_M_1.jpg?$Large$',
 'https://images.topshop.com/i/Top

In [140]:
len(links)

363

In [141]:
desc

['NYC Check Sequin Backpack',
 'IBIZA Snake Sporty Cross Body Bag',
 'BLISS Chartreuse Crocodile Bumbag',
 'NYC Check Sequin Bumbag',
 'IBIZA Crinkle Bag',
 'SEANNA Transparent PU Clear Gingham Tote Bag',
 'SYDNEY Black Straw Bowler Bag',
 'SYDNEY Straw Bowler Bag',
 'BEAR Wooden Grab Bag',
 "**You're My Lobster Cora Cross Body Bag by Skinnydip",
 '**Daisy Icon Make Up Bag by Skinnydip',
 '**Ffifon Gingham Cross Body Bag by Skinnydip',
 '**Tropical Fruit Washbag by Skinnydip',
 '**Lyla Pink Cross Body Bag by Skinnydip',
 '**Kaia Straw Tote Bag by Skinnydip',
 '**Casey Checked Cross Body Bag by Skinnydip',
 '**Lacey Gingham Cross Body Bag by Skinnydip',
 '**Freya Cat Tote Bag by Skinnydip',
 '**Ffifon Python Cross Body Bag by Skinnydip',
 '**Ffifon Lemon Drop Cross Body Bag by Skinnydip',
 'MABLE Gingham Heart Mini Bag',
 '**Lacey Strawberry Cross Body Bag by Skinnydip',
 '**Gemini Ditsy Cross Body Bag by Skinnydip',
 '**Envelope Faux Leather Bumbag by Koko Coture',
 '**Ffifon Croc Cros

In [142]:
len(desc)

363

In [143]:
prices

['$40.00',
 '$35.00',
 '$35.00',
 '$40.00',
 '$40.00',
 '$48.00',
 '$52.00',
 '$52.00',
 '$60.00',
 '$48.00',
 '$30.00',
 '$60.00',
 '$32.00',
 '$52.00',
 '$68.00',
 '$56.00',
 '$60.00',
 '$52.00',
 '$60.00',
 '$60.00',
 '$40.00',
 '$60.00',
 '$64.00',
 '$36.00',
 '$60.00',
 '$38.00',
 '$56.00',
 '$60.00',
 '$60.00',
 '$40.00',
 '$60.00',
 '$64.00',
 '$36.00',
 '$60.00',
 '$38.00',
 '$56.00',
 '$36.00',
 '$34.00',
 '$52.00',
 '$36.00',
 '$28.00',
 '$52.00',
 '$52.00',
 '$32.00',
 '$52.00',
 '$68.00',
 '$68.00',
 '$48.00',
 '$55.00',
 '$48.00',
 '$40.00',
 '$40.00',
 '$35.00',
 '$35.00',
 '$68.00',
 '$68.00',
 '$48.00',
 '$55.00',
 '$48.00',
 '$40.00',
 '$40.00',
 '$35.00',
 '$35.00',
 '$48.00',
 '$52.00',
 '$52.00',
 '$75.00',
 '$48.00',
 '$36.00',
 '$52.00',
 '$52.00',
 '$40.00',
 '$48.00',
 '$52.00',
 '$70.00',
 '$52.00',
 '$40.00',
 '$48.00',
 '$68.00',
 '$68.00',
 '$75.00',
 '$48.00',
 '$52.00',
 '$70.00',
 '$52.00',
 '$40.00',
 '$48.00',
 '$68.00',
 '$68.00',
 '$75.00',
 '$48.00',

In [144]:
len(prices)

363

In [174]:
links

['https://images.topshop.com/i/TopShop/TS24P05QMON_M_1.jpg?$Large$',
 'https://images.topshop.com/i/TopShop/TS24R14QNAT_M_1.jpg?$Large$',
 'https://images.topshop.com/i/TopShop/TS24I13QCHT_M_1.jpg?$Large$',
 'https://images.topshop.com/i/TopShop/TS24I14QMON_M_1.jpg?$Large$',
 'https://images.topshop.com/i/TopShop/TS24I08QRED_M_1.jpg?$Large$',
 'https://images.topshop.com/i/TopShop/TS24W15QCLR_M_1.jpg?$Large$',
 'https://images.topshop.com/i/TopShop/TS24B03QBLK_M_1.jpg?$Large$',
 'https://images.topshop.com/i/TopShop/TS24B03QNAT_M_1.jpg?$Large$',
 'https://images.topshop.com/i/TopShop/TS24B32QNAT_M_1.jpg?$Large$',
 'https://images.topshop.com/i/TopShop/TS63Q30QMUL_M_1.jpg?$Large$',
 'https://images.topshop.com/i/TopShop/TS63H91PMUL_M_1.jpg?$Large$',
 'https://images.topshop.com/i/TopShop/TS63Q23QMUL_F_1.jpg?$Large$',
 'https://images.topshop.com/i/TopShop/TS63H90PMUL_M_1.jpg?$Large$',
 'https://images.topshop.com/i/TopShop/TS63Q12QPNK_M_1.jpg?$Large$',
 'https://images.topshop.com/i/Top

In [178]:
links2 = [l.replace("_M_", "_F_") if "_M_" in l else l for l in links ]

In [179]:
len(links2)

363

In [180]:
all_data = []

for i in range(len(links)):
    all_data.append({'description': desc[i],
               'link': links2[i],
               'price': prices[i],
               'brand': 'Topshop'})

In [181]:
df_data = pd.DataFrame(all_data)

In [182]:
df_data

,brand,description,link,price
0,Topshop,NYC Check Sequin Backpack,https://images.topshop.com/i/TopShop/TS24P05QM...,$40.00
1,Topshop,IBIZA Snake Sporty Cross Body Bag,https://images.topshop.com/i/TopShop/TS24R14QN...,$35.00
2,Topshop,BLISS Chartreuse Crocodile Bumbag,https://images.topshop.com/i/TopShop/TS24I13QC...,$35.00
3,Topshop,NYC Check Sequin Bumbag,https://images.topshop.com/i/TopShop/TS24I14QM...,$40.00
4,Topshop,IBIZA Crinkle Bag,https://images.topshop.com/i/TopShop/TS24I08QR...,$40.00
5,Topshop,SEANNA Transparent PU Clear Gingham Tote Bag,https://images.topshop.com/i/TopShop/TS24W15QC...,$48.00
6,Topshop,SYDNEY Black Straw Bowler Bag,https://images.topshop.com/i/TopShop/TS24B03QB...,$52.00
7,Topshop,SYDNEY Straw Bowler Bag,https://images.topshop.com/i/TopShop/TS24B03QN...,$52.00
8,Topshop,BEAR Wooden Grab Bag,https://images.topshop.com/i/TopShop/TS24B32QN...,$60.00
9,Topshop,**You're My Lobster Cora Cross Body Bag by Ski...,https://images.topshop.com/i/TopShop/TS63Q30QM...,$48.00


In [183]:
df_data.drop_duplicates(subset = 'link', inplace = True)

In [184]:
df_data.reset_index(inplace = True)
df_data.head()

,index,brand,description,link,price
0,0,Topshop,NYC Check Sequin Backpack,https://images.topshop.com/i/TopShop/TS24P05QM...,$40.00
1,1,Topshop,IBIZA Snake Sporty Cross Body Bag,https://images.topshop.com/i/TopShop/TS24R14QN...,$35.00
2,2,Topshop,BLISS Chartreuse Crocodile Bumbag,https://images.topshop.com/i/TopShop/TS24I13QC...,$35.00
3,3,Topshop,NYC Check Sequin Bumbag,https://images.topshop.com/i/TopShop/TS24I14QM...,$40.00
4,4,Topshop,IBIZA Crinkle Bag,https://images.topshop.com/i/TopShop/TS24I08QR...,$40.00


In [185]:
df_data.columns

Index(['index', 'brand', 'description', 'link', 'price'], dtype='object')

In [186]:
df_data.drop(columns = ['index'], inplace = True)
df_data

,brand,description,link,price
0,Topshop,NYC Check Sequin Backpack,https://images.topshop.com/i/TopShop/TS24P05QM...,$40.00
1,Topshop,IBIZA Snake Sporty Cross Body Bag,https://images.topshop.com/i/TopShop/TS24R14QN...,$35.00
2,Topshop,BLISS Chartreuse Crocodile Bumbag,https://images.topshop.com/i/TopShop/TS24I13QC...,$35.00
3,Topshop,NYC Check Sequin Bumbag,https://images.topshop.com/i/TopShop/TS24I14QM...,$40.00
4,Topshop,IBIZA Crinkle Bag,https://images.topshop.com/i/TopShop/TS24I08QR...,$40.00
5,Topshop,SEANNA Transparent PU Clear Gingham Tote Bag,https://images.topshop.com/i/TopShop/TS24W15QC...,$48.00
6,Topshop,SYDNEY Black Straw Bowler Bag,https://images.topshop.com/i/TopShop/TS24B03QB...,$52.00
7,Topshop,SYDNEY Straw Bowler Bag,https://images.topshop.com/i/TopShop/TS24B03QN...,$52.00
8,Topshop,BEAR Wooden Grab Bag,https://images.topshop.com/i/TopShop/TS24B32QN...,$60.00
9,Topshop,**You're My Lobster Cora Cross Body Bag by Ski...,https://images.topshop.com/i/TopShop/TS63Q30QM...,$48.00


In [187]:
num = list(range(len(df_data)))

In [188]:
df_data = pd.concat([df_data, pd.Series(num)], axis =1)

In [189]:
df_data.columns

Index(['brand', 'description', 'link', 'price', 0], dtype='object')

In [190]:
df_data['img-file'] = df_data[0].apply(lambda x: 'top_'+str(x+1)+'.png')

In [191]:
df_data.drop(columns = 0, inplace=True)
df_data

,brand,description,link,price,img-file
0,Topshop,NYC Check Sequin Backpack,https://images.topshop.com/i/TopShop/TS24P05QM...,$40.00,top_1.png
1,Topshop,IBIZA Snake Sporty Cross Body Bag,https://images.topshop.com/i/TopShop/TS24R14QN...,$35.00,top_2.png
2,Topshop,BLISS Chartreuse Crocodile Bumbag,https://images.topshop.com/i/TopShop/TS24I13QC...,$35.00,top_3.png
3,Topshop,NYC Check Sequin Bumbag,https://images.topshop.com/i/TopShop/TS24I14QM...,$40.00,top_4.png
4,Topshop,IBIZA Crinkle Bag,https://images.topshop.com/i/TopShop/TS24I08QR...,$40.00,top_5.png
5,Topshop,SEANNA Transparent PU Clear Gingham Tote Bag,https://images.topshop.com/i/TopShop/TS24W15QC...,$48.00,top_6.png
6,Topshop,SYDNEY Black Straw Bowler Bag,https://images.topshop.com/i/TopShop/TS24B03QB...,$52.00,top_7.png
7,Topshop,SYDNEY Straw Bowler Bag,https://images.topshop.com/i/TopShop/TS24B03QN...,$52.00,top_8.png
8,Topshop,BEAR Wooden Grab Bag,https://images.topshop.com/i/TopShop/TS24B32QN...,$60.00,top_9.png
9,Topshop,**You're My Lobster Cora Cross Body Bag by Ski...,https://images.topshop.com/i/TopShop/TS63Q30QM...,$48.00,top_10.png


In [203]:
df_data[df_data['img-file'] =='top_94.png']

,brand,description,link,price,img-file
93,Topshop,MIESHA Ball Handle Grab Bag,https://images.topshop.com/i/TopShop/TS24S17QG...,$52.00,top_94.png


In [204]:
df_data.drop(index = 93, axis = 0, inplace=True)

In [205]:
df_data[df_data['img-file'] =='top_94.png']

,brand,description,link,price,img-file


In [206]:
df_data.reset_index(inplace=True)

In [208]:
df_data.drop(columns='index', inplace=True)

In [209]:
df_data

,brand,description,link,price,img-file
0,Topshop,NYC Check Sequin Backpack,https://images.topshop.com/i/TopShop/TS24P05QM...,$40.00,top_1.png
1,Topshop,IBIZA Snake Sporty Cross Body Bag,https://images.topshop.com/i/TopShop/TS24R14QN...,$35.00,top_2.png
2,Topshop,BLISS Chartreuse Crocodile Bumbag,https://images.topshop.com/i/TopShop/TS24I13QC...,$35.00,top_3.png
3,Topshop,NYC Check Sequin Bumbag,https://images.topshop.com/i/TopShop/TS24I14QM...,$40.00,top_4.png
4,Topshop,IBIZA Crinkle Bag,https://images.topshop.com/i/TopShop/TS24I08QR...,$40.00,top_5.png
5,Topshop,SEANNA Transparent PU Clear Gingham Tote Bag,https://images.topshop.com/i/TopShop/TS24W15QC...,$48.00,top_6.png
6,Topshop,SYDNEY Black Straw Bowler Bag,https://images.topshop.com/i/TopShop/TS24B03QB...,$52.00,top_7.png
7,Topshop,SYDNEY Straw Bowler Bag,https://images.topshop.com/i/TopShop/TS24B03QN...,$52.00,top_8.png
8,Topshop,BEAR Wooden Grab Bag,https://images.topshop.com/i/TopShop/TS24B32QN...,$60.00,top_9.png
9,Topshop,**You're My Lobster Cora Cross Body Bag by Ski...,https://images.topshop.com/i/TopShop/TS63Q30QM...,$48.00,top_10.png


In [210]:
df_data.to_csv('topshop_data.csv')

### 2. Loop through image links and save as a .png file

In [193]:
# loop through the links to edit url before requesting and converting image url to png file and save locally
for i, url in enumerate(df_data.link):
    time.sleep(1)
    try:
        request = urllib.request.Request(url)
        response = urllib.request.urlopen(request)
        binary_str = response.read()
        byte_array = bytearray(binary_str)
        numpy_array = np.asarray(byte_array, dtype='int8')
        image = cv2.imdecode(numpy_array, cv2.IMREAD_UNCHANGED)
        cv2.imwrite("Topshop2/"+"top_{}".format(i+1)+".png", image)
        print("Saved "+"top_{}".format(i+1)+".png")
    except Exception as e:
        print(str(e))

print("Done")

Saved top_1.png
Saved top_2.png
Saved top_3.png
Saved top_4.png
Saved top_5.png
Saved top_6.png
Saved top_7.png
Saved top_8.png
Saved top_9.png
Saved top_10.png
Saved top_11.png
Saved top_12.png
Saved top_13.png
Saved top_14.png
Saved top_15.png
Saved top_16.png
Saved top_17.png
Saved top_18.png
Saved top_19.png
Saved top_20.png
Saved top_21.png
Saved top_22.png
Saved top_23.png
Saved top_24.png
Saved top_25.png
Saved top_26.png
Saved top_27.png
Saved top_28.png
Saved top_29.png
Saved top_30.png
Saved top_31.png
Saved top_32.png
Saved top_33.png
Saved top_34.png
Saved top_35.png
Saved top_36.png
Saved top_37.png
Saved top_38.png
Saved top_39.png
Saved top_40.png
Saved top_41.png
Saved top_42.png
Saved top_43.png
Saved top_44.png
Saved top_45.png
Saved top_46.png
Saved top_47.png
Saved top_48.png
Saved top_49.png
Saved top_50.png
Saved top_51.png
Saved top_52.png
Saved top_53.png
Saved top_54.png
Saved top_55.png
Saved top_56.png
Saved top_57.png
Saved top_58.png
Saved top_59.png
Saved 